In [30]:
import pandas as pd
import numpy as np
import sklearn as sk

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

# Load dataset
titanic = pd.read_csv("titanic.csv", header=0, sep=",")

# Normalize dataset
# Normalize sex
titanic['Sex'].replace(['female','male'],[0,1],inplace=True)
# Normalize parents - if needed
# titanic = titanic[titanic['Parents'] > 2]
#titanic['Parents'] = std_scaler.fit_transform(titanic[['Parents']])
# Normalize siblings
#titanic['Siblings'] = std_scaler.fit_transform(titanic[['Siblings']])
# Normalize age
titanic['Age'] = std_scaler.fit_transform(titanic[['Age']])
# Normalize passenger class
p_classes = pd.get_dummies(titanic.Pclass, prefix='Class')
titanic = titanic.join(p_classes).drop(['Pclass'], axis=1)
# Normalize fare
titanic['Fare'] = std_scaler.fit_transform(titanic[['Fare']])

print(titanic.describe())

titanic = sk.utils.shuffle(titanic)

# Split dataset to train, test and valid
train, test = np.split(titanic, [int(.7*len(titanic))])
# train, test, valid = np.split(titanic, [int(.7*len(titanic)), int(.9*len(titanic))])

train_labels = train['Survived']
train_data = train.drop(['Survived', 'Name'], axis=1)
test_labels = test['Survived']
test_data = test.drop(['Survived', 'Name'], axis=1)
# valid_labels = valid['Survived']
# valid_data = valid.drop(['Survived', 'Name'], axis=1)

         Survived         Sex           Age    Siblings     Parents  \
count  887.000000  887.000000  8.870000e+02  887.000000  887.000000   
mean     0.385569    0.645998  2.403189e-17    0.525366    0.383315   
std      0.487004    0.478480  1.000564e+00    1.104669    0.807466   
min      0.000000    0.000000 -2.058350e+00    0.000000    0.000000   
25%      0.000000    0.000000 -6.533568e-01    0.000000    0.000000   
50%      0.000000    1.000000 -1.042546e-01    0.000000    0.000000   
75%      1.000000    1.000000  6.042645e-01    1.000000    0.000000   
max      1.000000    1.000000  3.580045e+00    8.000000    6.000000   

               Fare     Class_1     Class_2     Class_3  
count  8.870000e+02  887.000000  887.000000  887.000000  
mean   1.301727e-16    0.243517    0.207441    0.549042  
std    1.000564e+00    0.429447    0.405703    0.497870  
min   -6.493034e-01    0.000000    0.000000    0.000000  
25%   -4.900196e-01    0.000000    0.000000    0.000000  
50%   -3.587

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F



# Hyperparameters
input_size = 6
output_size = 2
hidden_size = 200
epochs = 20
batch_size = 50
learning_rate = 0.00003

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l3(x)
        return F.log_softmax(x)

# Convert data to tensors
# x_ax = torch.FloatTensor(train_data.tolist())
# y_ax = torch.LongTensor(train_labels.tolist())

net = Network()
print(net)

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [ ]:
import tensorflow